<a href="https://colab.research.google.com/github/djurjo/DABI/blob/main/PSO-project/pso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The code


In [ ]:
!pip install numba

In [ ]:
from numba import jitclass, types, typed, njit, prange
from numba import int32, float32
import random
import numpy as np
import time, csv, math

Instead of creating a Bird class in the same way we do in te iterative version we will create some explicit parallel functions.

In [ ]:
#First we will need to create random arrays with the proper type
@njit
def create_random_array(list_intervals):
    size = len(list_intervals)
    r = np.zeros(size, dtype = np.float32)
    for i in range(0, size):
      b = random.random() # We use random randoms instead of np.randoms
      interval = list_intervals[i]
      b = (interval[1] - interval[0]) * b + interval[0]
      r[i] += b
    return r

# Some functions to operate with vectors
@njit
def add(v1, v2):
  r = np.zeros(v1.size, dtype = np.float32)
  if v1.size == v2.size:
    for i in range(0, v1.size):
      r[i] = v1[i] + v2[i]
  return r

@njit
def escalar(v1, val):
  r = np.zeros(v1.size, dtype = np.float32)
  for i in range(v1.size):
    r[i] = v1[i]* val
  return r

@njit
def sub(v1, v2):
  r = np.zeros(v1.size, dtype = np.float32)
  if v1.size == v2.size:
    for i in range(0, v1.size):
      r[i] = v1[i] - v2[i]
  return r

So we have:


*   **Swarn**: An N dimensional Array of Birds
*   **Bird**: An 3xSize dimensional Array. Such that:
    * Bird[0] := Current Position
    * Bird[1] := Current Vector
    * Bird[2] := Best local position
 



In [ ]:
def create_swarn(search_space, B):
  swarn = []
  for i in prange(B):
    position = create_random_array(search_space)
    direction = create_random_array(search_space)
    b = np.vstack((position, direction, position))
    swarn.append(b)
  t_swarn = typed.List()    
  [t_swarn.append(x) for x in swarn]   
  return t_swarn

def update_swarn_vector(swarn, best_global_postion, cog, soc, inertia, r1, r2):
  for i in prange(len(swarn)):
    bird = swarn[i]
    s1 = escalar(bird[1], inertia)
    s2 = escalar(sub(bird[2], bird[0]) , cog*r1)
    s3 = escalar(sub(best_global_postion, bird[0]) , soc*r2)
    swarn[i][1] = add(add(s1, s2), s3)

In [ ]:
@njit
def in_search_space(current_pos, search_space):
    for i in range(len(search_space)):
        if (search_space[i][0] <= current_pos[i]) and (current_pos[i] <= search_space[i][1]):
           pass
        else:
            if (abs(search_space[i][0] - current_pos[i]) < abs(search_space[i][1] - current_pos[i])):
                current_pos[i] = current_pos[i] % search_space[i][0]
            else:
                current_pos[i] = current_pos[i] % search_space[i][1]
    return current_pos


@njit
def move_swarn(swarn, search_space):
  for i in range(len(swarn)):
    bird = swarn[i]
    n_pos = add(bird[0], bird[1])
    swarn[i][0] = in_search_space(n_pos, search_space)


In [ ]:
def update_best_local(swarn, f):
  for i in prange(len(swarn)):
    bird = swarn[i]
    if f(bird[0]) < f(bird[2]):
      swarn[i][2] = bird[0]

def take_best_global(swarn, f):
  b_g_p = swarn[0][2]
  b_g_v = f(b_g_p)
  for i in prange(len(swarn)):
    bird = swarn[i]
    if f(bird[2]) < b_g_v :
      b_g_p = bird[2]
      b_g_v = f(bird[2])
    else:
      pass
  return (b_g_p, b_g_v)

In [ ]:
@njit
def move_bird(bird, search_space):
    n_pos = add(bird[0], bird[1])
    bird[0] = in_search_space(n_pos, search_space)
    return bird

@njit
def bird_update_local(bird, f):
    if f(bird[0]) < f(bird[2]):
        bird[2] = bird[0]
    return bird

@njit
def update_bird(bird, best_global_postion, cog, soc, inertia, r1, r2):
    s1 = escalar(bird[1], inertia)
    s2 = escalar(sub(bird[2], bird[0]) , cog*r1)
    s3 = escalar(sub(best_global_postion, bird[0]) , soc*r2)
    bird[1] = add(add(s1, s2), s3)
    return bird

@njit
def bird_upd(bird, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
    bird = update_bird(bird, b_g_p, cog, soc, inertia ,r1, r2)
    bird = move_bird(bird, search_space)
    bird = bird_update_local(bird, f)
    return bird


@njit(parallel=True)
def swarn_upd_par(swarn, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
  for j in prange(0, len(swarn)):
    swarn[j] = bird_upd(swarn[j], b_g_p, cog, soc, inertia ,r1, r2, search_space, f)
  return swarn

def swarn_upd_itr(swarn, b_g_p, cog, soc, inertia ,r1, r2, search_space, f):
  for j in prange(0, len(swarn)):
    swarn[j] = bird_upd(swarn[j], b_g_p, cog, soc, inertia ,r1, r2, search_space, f)
  return swarn


In [ ]:
def run_pso(iterations, B, space, target, cog, soc, inertia, r1, r2, parallel):
  swarn = create_swarn(space,B)
  t = take_best_global(swarn, target)
  b_g_p = t[0]
  b_g_v = t[1]
  if parallel:
    for it in range(1, iterations + 1):
      swarn = swarn_upd_par(swarn, b_g_p, cog, soc, inertia ,r1, r2, space, target)
      t = take_best_global(swarn, target)
      b_g_p = t[0]
      b_g_v = t[1]
  else:
    for it in range(1, iterations + 1):
      swarn = swarn_upd_itr(swarn, b_g_p, cog, soc, inertia ,r1, r2, space, target)
      t = take_best_global(swarn, target)
      b_g_p = t[0]
      b_g_v = t[1]
      
  return (b_g_v, b_g_p)

In [ ]:
#### TEST FUNCTIONS ####

@njit
def sphere(x):
  s = 0
  for i in prange(len(x)):
    s += x[i]**2
  return s

def sphere2(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += x[i]**2
  return s


@njit
def rosenbrock(x):
  s = 0
  for i in prange(len(x)-1):
    s += 100*(x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
  return s

def rosenbrock2(x):
  s = 0
  x = list(x)
  for i in range(len(x)-1):
    s += 100*(x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
  return s

@njit 
def styblinski(x):
  s = 0
  for i in prange(len(x)):
    s += (x[i]**4) - 16*(x[i]**2) + 5*x[i]
  return s/2

def styblinski2(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += (x[i]**4) - 16*(x[i]**2) + 5*x[i]
  return s/2

def minStyb(dimension):
  sol = [-2.903534]*dimension
  solsty = typed.List()
  [solsty.append(x) for x in sol]
  return styblinski2(solsty)

### min between -39.16617*n < f(-2.903534, ... , -2.903534)<-39.16616*n 

@njit
def f8(x):
  s = 0
  for i in prange(len(x)):
    s += (- x[i]) * (math.sin(math.sqrt(abs(x[i]))))
  return s

def f82(x):
  s = 0
  x = list(x)
  for i in range(len(x)):
    s += (- x[i]) * (math.sin(math.sqrt(abs(x[i]))))
  return s

## min  [-500, 500] is -12569.5

@njit 
def f2(x):
  s = 0
  p = 0
  for i in range(len(x)):
    s += abs(x[i])
    p *= abs(x[i])
  return s + p

def f22(x):
  s = 0
  p = 0
  x = list(x)
  for i in range(len(x)):
    s += abs(x[i])
    p *= abs(x[i])
  return s + p

@njit
def u(x, a, k, m):
  if x > a:
    return k*((x - a)**m)
  elif -a <= x and x <= a:
    return 0
  else:
    return k*((- x - a)**m)

def y(x):
  return 1 + (1/4)*(x + 1)

@njit
def f13(x):
  s1 = 0
  s2 = 0
  for i in range(len(x)):
    if i < len(x) - 1:
      s1 += ((x[i] - 1)**2) * (1 + (math.sin(3 * math.pi * x[i+1]))**2) 
    s2 += u(x[i], 5, 100, 4)
  return 0.1*((math.sin(3 * math.pi * x[1]))**2 + s1 + (x[len(x) - 1] - 1)*(1 + (math.sin(2 * math.pi * x[len(x) - 1]))**2)) + s2

def f132(x):
  s1 = 0
  s2 = 0
  x = list(x)
  for i in range(len(x)):
    if i < len(x) - 1:
      s1 += ((x[i] - 1)**2) * (1 + (math.sin(3 * math.pi * x[i+1]))**2) 
    s2 += u(x[i], 5, 100, 4)
  return 0.1*((math.sin(3 * math.pi * x[1]))**2 + s1 + (x[len(x) - 1] - 1)*(1 + (math.sin(2 * math.pi * x[len(x) - 1]))**2)) + s2



target_functions = { # id : (function, space, global_min, function2)
    1 : (sphere, [(-500, 500)]*5, 0, sphere2),
    2 : (rosenbrock, [(-100, 100)]*5, 0, rosenbrock2),
    3 : (styblinski, [(-5, 5)]*5, minStyb(5), styblinski2), 
    4 : (f8, [(-500, 500)]*5, -12569.5, f82),
    5 : (f2, [(-10, 10)]*5, 0, f22),
    6 : (f13, [(-50, 50)]*5, 0, f132)
}
names = {
    1 : 'Sphere',
    2 : 'Rosenbrock',
    3 : 'Styblinski', 
    4 : 'f8', 
    5 : 'f2',
    6 : 'f13'
}


In [ ]:
#### Global variables
runs = 30

cog = 1.7  # Cognitive importance
soc = 2.3   # Social importance
inertia = 0.6

# Two random values that will change each time
r1 = random.random() 
r2 = random.random()

# Number of birds
B = 100

# Number of iterations

iterations = 1000

pairs = [(100, 1000), (200, 1000), (500, 1000), (100, 5000), (500, 5000)]


# A small example


In [ ]:
for key in target_functions.keys():
  (target, space, g_min, target2) = target_functions[key]
  t_space = typed.List()
  for x in space:
    t_space.append(x)
  s_par = run_pso(iterations, B, t_space, target, cog, soc, inertia, r1, r2, True)
  s_itr = run_pso(iterations, B, t_space, target, cog, soc, inertia, r1, r2, False)
  print("Function: ", names[key], " with parallel: ", s_par[0], " real min ", g_min)
  print("Function: ", names[key], " with iterative: ", s_itr[0]," real min ", g_min)

Function:  Sphere  with parallel:  0.010295355576090515  real min  0
Function:  Sphere  with iterative:  0.5632673953077756  real min  0
Function:  Rosenbrock  with parallel:  4.106022839395958  real min  0
Function:  Rosenbrock  with iterative:  4.670636039392463  real min  0
Function:  Styblinski  with parallel:  -181.6854430437088  real min  -195.830828518857
Function:  Styblinski  with iterative:  -167.03602027893066  real min  -195.830828518857
Function:  f8  with parallel:  -1494.8707885742188  real min  -12569.5
Function:  f8  with iterative:  -1607.9533386230469  real min  -12569.5
Function:  f2  with parallel:  0.0010737266014708147  real min  0
Function:  f2  with iterative:  9.327483260334474e-08  real min  0
Function:  f13  with parallel:  -1.0305935188735016  real min  0
Function:  f13  with iterative:  -1.1384220443359554  real min  0


# The Experiment

In [ ]:


#### This cell runs the experiments and record all the data in a csv file.

### We run the pso 40 times for each function and type (parallel or not) 
### and we record the mean time per run and the absolute mean error.

### First we are going to run all the functions and the pso in order to initialize the @njit functions

p = typed.List()
[p.append(x) for x in [0, 1 , 0, 1, -1, 1, 0, 1]]

for k in target_functions:
    x = target_functions[k][0](p)

t_sp = typed.List()
t_sp.append((-1, 1)) 
x = run_pso(10, 10, t_sp, sphere, cog, soc, inertia, r1, r2, True)


print('Experiment starts: ')

with open('experiments_results.csv', mode='w') as experiment:
  exp = csv.writer(experiment, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  exp.writerow(['N. Birds', 'Iterations', 'Run', 'Target function', 'Type', 'Error', 'Point', 'Time (s)'])

  for (iterations, B) in pairs:
    for key in target_functions.keys():
      (target, space, g_min, target2) = target_functions[key]
      t_space = typed.List()
      for x in space:
        t_space.append(x)
      
      total_time_par = 0
      total_error_par = 0

      total_time_itr = 0
      total_error_itr = 0

      for run in range(runs):
        
        time_par = time.time()
        solution = run_pso(iterations, B, t_space, target, cog, soc, inertia, r1, r2, True)
        (solution_par, point_par) = (solution[0], solution[1])
        time_t_par = time.time() - time_par
        corrector = target2(point_par)
        
        if (solution_par - corrector > 1):
          print("Big error detected. Njit calculated solution: ", solution_par, ". Python solution: ", corrector)
          print("Point: ", point_par)
          solution_par = corrector

        error_par = abs(g_min - corrector)
        total_time_par = total_time_par + time_t_par
        total_error_par = total_error_par + solution_par
        
        #print("Parallel: ", str(time_t_par), " error: ", total_error_par, " point: ", point_par)

        exp.writerow([B, iterations, str(run + 1), names[key], 'Parallel',error_par, point_par, time_t_par])

        time_itr = time.time()
        solution = run_pso(iterations, B, t_space, target, cog, soc, inertia, r1, r2, False)
        (solution_itr, point_itr) = (solution[0], solution[1])
        time_t_itr = time.time() - time_itr 
        corrector = target2(point_itr)
        
        if (solution_itr - corrector > 1):
          print("Big error detected. Njit calculated solution: " , solution_itr,"Python solution: " ,corrector)
          print("Point: ", point_itr)
          solution_itr = corrector

        error_itr = abs(g_min - solution_itr)
        total_time_itr = total_time_itr + time_t_itr
        total_error_itr = total_error_itr + error_itr

        #print("Iterative: ", str(time_t_itr), " error: ", total_error_itr, " point: ", point_itr)

        exp.writerow([B, iterations, str(run + 1), names[key], 'Iterative', error_itr, point_itr, time_t_itr])

        r1 = random.random()
        r2 = random.random()

      print('The experiment ', names[key], ' ended with: ')
      print('Parallel ---- ', str(total_time_par), ' seconds with a mean error of ', str(total_error_par/runs))
      print('Iterative ---- ', str(total_time_itr), ' seconds with a mean error of ', str(total_error_itr/runs))



Experiment starts: 
The experiment  Sphere  ended with: 
Parallel ----  14.343066692352295  seconds with a mean error of  182.10097883749538
Iterative ----  73.21604108810425  seconds with a mean error of  166.6117983900895
The experiment  Rosenbrock  ended with: 
Parallel ----  15.919832944869995  seconds with a mean error of  2377.333559282991
Iterative ----  70.34689450263977  seconds with a mean error of  403.9597379616412
The experiment  Styblinski  ended with: 
Parallel ----  16.112332820892334  seconds with a mean error of  -191.35041069189708
Iterative ----  70.35131120681763  seconds with a mean error of  6.285858595528794
The experiment  f8  ended with: 
Parallel ----  17.52836275100708  seconds with a mean error of  -1784.824905649821
Iterative ----  72.81688666343689  seconds with a mean error of  10813.826969655354
The experiment  f2  ended with: 
Parallel ----  16.050729036331177  seconds with a mean error of  0.2302988049326374
Iterative ----  71.40649390220642  seconds 